In [50]:
import numpy as np
import pandas as pd
import string
import nltk
import threading # will potentially use multi-threading
from nltk.stem.porter import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet

In [32]:
df_docs = pd.read_csv('Documents_Complete.csv')

In [33]:
df_docs

,body,places,title,topics
0,"said, expect, earlier, report, remov, jone, in...",canada,"major, impact, remov",NaN
1,"mason, former, presid, chief, oper, offic, emp...",usa,"former, empir, carolina, exec, sentenc",NaN
2,"discoveri, complic, search, vaccin, team, armi...",usa,"doctor, find, link, smallpox, viru",NaN
3,"doctor, center, diseas, control, atlanta, said...",usa,"birth, control, pill, help, prevent, cancer, s...",NaN
4,"econom, data, week, determin, interest, rate, ...",usa,"econom, data, debt, futur, outlook","interest, retail, ipi"
5,"reagan, administr, respond, last, year, unit, ...",usa,"action, program, africa",NaN
6,"novel, type, financ, texa, instrument, market,...",usa,"unusu, texa, instrument, prefer, price",NaN
7,"canadian, foreign, secretari, clark, wind, vis...","usa, canada","clark, expect, action, acid, rain",NaN
8,"ford, motor, said, check, distribut, employe, ...",usa,"ford, motor, distribut, profit, share",NaN
9,"jamaica, billion, foreign, debt, reduc, oblig,...",jamaica,"jamaica, borrow",NaN


**Removing documents with no Topics, no Body and no Title**

In [34]:

df_filter = df_docs[df_docs['topics'].notnull() & df_docs['body'].notnull() & df_docs['title'].notnull()]


In [35]:
count = df_filter['body'].str.split().str.len()
df_filter = df_filter[~(count<200)]
df_filter.reset_index()
df_filter

,body,places,title,topics
4,"econom, data, week, determin, interest, rate, ...",usa,"econom, data, debt, futur, outlook","interest, retail, ipi"
25,"hous, trade, lawmak, took, first, vote, measur...",usa,"hous, panel, take, first, trade, bill, vote",trade
48,"intern, effort, redirect, japan, economi, dome...","japan, usa","econom, spotlight, heavi, fight, back","ship, iron-steel"
66,"saudi, arabia, commerci, bank, report, declin,...",saudi-arabia,"saudi, bank, face, lean, period",earn
70,"indonesian, coffe, export, prepar, period, dep...","indonesia, costa-rica, dominican-republic, hon...","indonesia, press, compromis, coffe",coffee
92,"taiwan, control, currenc, inflow, implement, t...","taiwan, usa","taiwan, clamp, control, currenc, inflow","money-fx, reserves"
100,"australia, expect, meat, livestock, export, mi...","australia, bahrain, uae, saudi-arabia, kuwait,...","australia, expect, rise, meat, export, mideast","carcass, livestock"
164,"worst, japan, batter, economi, economist, said...",japan,"japan, economi, near, bottom, economist",gnp
173,"japanes, compani, decid, renew, term, contract...","japan, iran","japan, firm, renew, iran, term, contract",crude
222,"mart, corp, said, record, fiscal, earn, rise, ...",usa,"mart, record, turn, point",earn


**The number of documents after preprocessing are 513**

# Data Visualization

In [56]:
# Function to get top n words from any columns
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

**Group by Topics**

In [47]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


df_filter['topics'].iplot(kind='hist', bins=100, xTitle='Topics',linecolor='black', yTitle='count', title=' Distribution')



In [59]:
common_words = get_top_n_words(df_filter['topics'], 20)
#for word, freq in common_words:
#    print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['TopicText' , 'count'])
df2.groupby('TopicText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 Topics')

**Group by Places**

In [48]:
df_filter.groupby('places').count()['body'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', opacity=0.8,
                                                           title='Bar chart of Distribution of places', xTitle='Places')

In [67]:
df_places_no_nan = df_filter['places'].dropna()

common_words = get_top_n_words(df_places_no_nan, 20)
#for word, freq in common_words:
    #print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['PlaceText' , 'count'])
df2.groupby('PlaceText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in Places')

**Group by Title**

In [69]:
common_words = get_top_n_words(df_filter['title'], 20)
#for word, freq in common_words:
#    print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['TitleText' , 'count'])
df2.groupby('TitleText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in Title')